# Classificação de Imagens com Redes Neurais


É importante lembrar que toda solução emerge a partir de um problema identificado. Podemos pensar na seguinte situação: Sofia tem uma orta no seu quintal com várias plantas, mas não tem tempo de olhar sempre se todas estão saudáveis. Então, depois de aprender na escola sobre inteligência artificial, ela teve a ideia de criar uma rede neural identificar se a planta estava doente ou saudável usando a mesma percepção que ela tinha para tomar essa decisão, que era olhar se havia manchas nas folhas. Durante 2 meses ela tirou fotos da folhas saudáveis e das folhas doentes e usou essas imagens para treinar a rede neural. Ela então usou essa rede neural treinada, junto com uma câmera instalada na orta para criar o sistema de monitoramento.

Ela percebeu que o sistema além de ajudar a monitorar quando ela estivesse ocupada, ele identifica muito antes do que ela o aparecimento das manchas nas folhas. Agora vamos implementar essse sistema usando a linguagem de programação Python e o Tensorflow.

#### Conectar com a pasta do google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### Importar pacotes

In [0]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle 
import tensorflow as tf 

## Pré-processamento

In [0]:
def importarPlantas(caminho, classificacao):
  ''' funcao importarPlantas
      caminho: caminho ate a pasta das imagens
      classificacao: acesso a subpasta 'doentes' ou 'saudaveis'
  '''
  # ler imagens na pasta
  imagens = os.listdir(caminho+classificacao)
  # loop para importar e preprocessar imagens
  caracteristicas=[]
  for imagem in imagens:
    img_array = cv2.imread(caminho+classificacao+'/'+imagem, cv2.IMREAD_GRAYSCALE) # ler arquivo em escala de cinza
    img_array = cv2.resize(img_array, (100,150), interpolation = cv2.INTER_AREA) # reduzir tamanho da imagem
    img_array = tf.keras.utils.normalize(img_array, axis=1) # normalizar array
    img_array = img_array.reshape(-1)        
    caracteristicas.append(img_array)
  # criar lista do alvo
  if classificacao == 'doentes':
    alvo = [1]*len(caracteristicas)
  elif classificacao == 'saudaveis':
    alvo = [0]*len(caracteristicas)
  # retornar valores
  return alvo, caracteristicas

In [0]:
# importar imagens
caminho = '/content/drive/My Drive/Projetos/Proj-Extensao-Programacao-IA-2020/Apresentacoes/Material-Aulas/Aula-3-Classificacao-RNA--Claudio-Estela-Gustavo/imagens_plantas/manga/'
doentes_alvo, doentes = importarPlantas(caminho, 'doentes')

In [0]:
saudaveis_alvo, saudaveis = importarPlantas(caminho, 'saudaveis')

In [0]:
# combinar doentes e saudaveis
import numpy as np
alvo =  np.asarray(doentes_alvo + saudaveis_alvo)
caracteristicas =  np.asarray(doentes + saudaveis)

In [0]:
# dividir em teste e treinamento
from sklearn.model_selection import train_test_split
caracteristicas_treino, caracteristicas_teste, alvo_treino, alvo_test = train_test_split(caracteristicas, alvo, test_size=0.2, random_state=89)

In [8]:
caracteristicas.shape

(370, 15000)

# Construção da Rede Neural

#### Camadas

In [9]:
import keras

model = keras.models.Sequential()
model.add(keras.layers.core.Dense(512, input_shape=tuple([caracteristicas.shape[1]]), activation='relu'))
model.add(keras.layers.core.Dense(360, activation='relu'))
model.add(keras.layers.core.Dense(360, activation='relu'))
model.add(keras.layers.core.Dense(64, activation='relu'))
model.add(keras.layers.core.Dense(2,   activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=["accuracy"])

print(model.summary())

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               7680512   
_________________________________________________________________
dense_2 (Dense)              (None, 360)               184680    
_________________________________________________________________
dense_3 (Dense)              (None, 360)               129960    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                23104     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 130       
Total params: 8,018,386
Trainable params: 8,018,386
Non-trainable params: 0
_________________________________________________________________
None


#### Treinar Testar o modelo

In [10]:
# train
model.fit(caracteristicas_treino, alvo_treino, epochs=30)

Epoch 1/30
296/296 [==============================] - 1s 3ms/step - loss: 0.8166 - accuracy: 0.5236
Epoch 2/30
296/296 [==============================] - 1s 2ms/step - loss: 0.7233 - accuracy: 0.5169
Epoch 3/30
296/296 [==============================] - 1s 2ms/step - loss: 0.7079 - accuracy: 0.5236
Epoch 4/30
296/296 [==============================] - 1s 2ms/step - loss: 0.7169 - accuracy: 0.5304
Epoch 5/30
296/296 [==============================] - 1s 2ms/step - loss: 0.6674 - accuracy: 0.6014
Epoch 6/30
296/296 [==============================] - 1s 2ms/step - loss: 0.6996 - accuracy: 0.5676
Epoch 7/30
296/296 [==============================] - 1s 2ms/step - loss: 0.6302 - accuracy: 0.6622
Epoch 8/30
296/296 [==============================] - 1s 2ms/step - loss: 0.6795 - accuracy: 0.5676
Epoch 9/30
296/296 [==============================] - 1s 2ms/step - loss: 0.6239 - accuracy: 0.6791
Epoch 10/30
296/296 [==============================] - 1s 2ms/step - loss: 0.6974 - accuracy: 0.5304

In [11]:
## avaliar
val_loss, val_acc = model.evaluate(caracteristicas_teste, alvo_test)

print(val_loss, val_acc)

74/74 [==============================] - 0s 622us/step
0.4398724758947218 0.7972972989082336
